In [15]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cv2
import os
import scipy.signal

from htr import page_detection
from htr import word_detection

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0)


def implt(img, cmp=None, t=''):
    plt.imshow(img, cmap=cmp)
    plt.title(t)
    plt.show()

In [16]:
def extract_and_save_chars(chars):
    count = 0
    for c in chars:
        cv2.imwrite("./training_data/test_set/" + str(count) + ".jpg", c)
        count += 1

In [17]:
path = "./data/slova/s4.jpg"
image = page_detection.detect(path)

characters = word_detection.detect(image)

processed = []
for c in characters:
    c_copy = c.copy()
    img = cv2.cvtColor(c_copy, cv2.COLOR_RGB2GRAY)
    img = 255 - img
    thresh = histogram_norm(img)
    
    cropped = cropp(thresh)
    
    resized = cv2.resize(255-cropped,(43,43))
    thresh = histogram_norm(resized)
    
    processed.append(resized)
    #implt(resized, 'gray')
    
extract_and_save_chars(processed)
print('DONE')

DONE


In [5]:
def bilateral_norm(img):
    img = cv2.bilateralFilter(img, 9, 15, 30)
    return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

def histogram_norm(img):
    img = bilateral_norm(img)
    add_img = 255 - cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    img = 255 - img
    img = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255        
    hist, bins = np.histogram(img.ravel(), 256, [0,256])
    
    img = img.astype(np.uint8)

    ret,thresh4 = cv2.threshold(img,np.argmax(hist)+10,255,cv2.THRESH_TOZERO)
    return add_img
    return cv2.add(add_img, thresh4, dtype=cv2.CV_8UC1)

In [4]:
def cropp(img):
    h,w = img.shape
    top=0
    down=0
    left=0
    right=0
    
    halt = False
    for i in range(h):
        if halt:
            break
        for j in range(w):
            if img[i,j] == 0:
                halt = True
                top = i-1
                break
                
    halt = False
    for i in reversed(range(h)):
        if halt:
            break
        for j in range(w):
            if img[i,j] == 0:
                halt = True
                down = i+1
                break
    
    halt = False
    for i in range(w):
        if halt:
            break
        for j in range(h):
            if img[j,i] == 0:
                halt = True
                left = i-1
                break
                
    halt = False
    for i in reversed(range(w)):
        if halt:
            break
        for j in range(h):
            if img[j,i] == 0:
                halt = True
                right = i+1
                break
                
    if (top < 0): top = 0
    if (down < 0): down = 0
    if (left < 0): left = 0
    if (right < 0): right = 0
        
    #print('Top: ', top)
    #print('Down: ', down)
    #print('Left: ', left)
    #print('Right: ', right)
    
    return img[top:down, left:right]